In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


In [ ]:

# 1. Download and load the model
model_id = "meta-llama/Llama-3.2-3B-Instruct"
#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:

# For Llama models, you'll need to use AutoModelForCausalLM instead of AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


In [ ]:

# 2. Save locally
local_directory = "./llama_3_2_3b_instruct"

model.save_pretrained(local_directory)
tokenizer.save_pretrained(local_directory)


In [ ]:

# 3. Load from local directory later
local_model = AutoModelForCausalLM.from_pretrained(local_directory)
local_tokenizer = AutoTokenizer.from_pretrained(local_directory)

# 4. Generate text with the model
inputs = tokenizer("Write a short poem about programming:", return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. Set up quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)

# 2. Download and load the model with quantization
model_id = "meta-llama/Llama-3.2-3B-Instruct"

# Authentication required
hf_token = "your_hf_token"  # Replace with your actual HuggingFace token

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=hf_token,
    quantization_config=quantization_config,
    device_map="auto"  # Automatically decide device placement
)

# 3. Save locally
local_directory = "./llama_3_2_3b_instruct_quantized"
model.save_pretrained(local_directory)
tokenizer.save_pretrained(local_directory)

# 4. Load from local directory later
# Note: When loading quantized models, you need to specify the quantization config again
local_model = AutoModelForCausalLM.from_pretrained(
    local_directory,
    quantization_config=quantization_config,
    device_map="auto"
)
local_tokenizer = AutoTokenizer.from_pretrained(local_directory)

# 5. Generate text with the model
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>
Write a short poem about programming.<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

inputs = tokenizer(prompt, return_tensors="pt").to(local_model.device)
outputs = local_model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))